In [ ]:
%matplotlib inline

In [ ]:
!nvidia-smi

In [ ]:
!lscpu

In [ ]:
%load_ext nb_black

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os
import sys
import time
import copy
import math
import pprint
import shutil
import zipfile
import random
import numpy as np
import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
from pathlib import Path

from PIL import Image
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

import torch
import torchvision
from torchvision import transforms, models
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset

from torchinfo import summary

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

mpl.style.use("seaborn")
sns.set_style("darkgrid")

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.metrics import confusion_matrix

import onnx
import tensorflow as tf
from onnx_tf.backend import prepare
from tensorflow.python.client import device_lib

tf.debugging.set_log_device_placement(True)

plt.rcParams.update({"figure.max_open_warning": 0})

In [ ]:
# Check GPU available for tf
print(tf.config.experimental.list_physical_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

In [ ]:
print("Is cuda available: {}".format(torch.cuda.is_available()))
current_device_id = torch.cuda.current_device()
device_info = torch.cuda.device(current_device_id)
devices_count = torch.cuda.device_count()
device_name = torch.cuda.get_device_name(0)
print(
    "Current device id is {} (there are {} devices in total)\ndevice info: {}\ndevice name: {}".format(
        current_device_id, devices_count, device_info, device_name, device_name
    )
)

In [ ]:
BATCH_SIZE = 64
EPOCHS_COUNT = 18
NUM_WORKERS = 16

IMAGE_CROP_WIDTH = 160
IMAGE_CROP_HEIGHT = 256

train_dir = "train"
val_dir = "val"

class_names = [
    "dab_left",
    "dab_right",
    "lottery_1",
    "lottery_2_left",
    "lottery_2_right",
    "say_so_1_left",
    "say_so_1_right",
    "wap_1_left",
    "wap_1_right",
    "wap_2",
    "wap_3_left",
    "wap_3_right",
    "wap_4_left",
    "wap_4_right",
]

In [ ]:
MODEL_SAVE_NAME = "dancer"
SUMBISSION_FILE_NAME = "submission"

PB_PATH = "./{}.pb".format(MODEL_SAVE_NAME)
TF_PATH = "./{}.tflite".format(MODEL_SAVE_NAME)

# --- DATASET ---

DATASET_DIR = "/data1/dataset"

# Train dataset
BOUNCED_DATASET_ZIP_DIR = "/data1/dataset/dataset_bounced.zip"
BOUNCED_DATASET_WORKING_DIR = "/data1/working_dataset_bounced/"

# Test dataset
TEST_DATASET_ZIP_DIR = "/data1/dataset/test_dataset.zip"
TEST_DATASET_WORKING_DIR = "/data1/working_dataset_bounced/"

# Dataset working dirs
DATA_ROOT = "/data1/working_dataset_bounced/dataset_bounced/"
TEST_DIR = "/data1/working_dataset_bounced/test/"

# --- PLOTS ---
RESULT_PLOTS_DIR = "plots"

# Data pre-processing

In [ ]:
print(os.listdir(DATASET_DIR))

In [ ]:
result_plots_dir_check = "./{}".format(RESULT_PLOTS_DIR)
confusion_matrix_plots_dir = "confusion_matrix"
confusion_matrix_plots_dir_check = "./{}/{}".format(
    RESULT_PLOTS_DIR, confusion_matrix_plots_dir
)

if not os.path.isdir(result_plots_dir_check):
    os.makedirs(result_plots_dir_check)
if not os.path.isdir(confusion_matrix_plots_dir_check):
    os.makedirs(confusion_matrix_plots_dir_check)

In [ ]:
with zipfile.ZipFile(BOUNCED_DATASET_ZIP_DIR, "r") as zip_obj:
    for member in tqdm(zip_obj.infolist(), desc="Extracting "):
        try:
            zip_obj.extract(member, BOUNCED_DATASET_WORKING_DIR)
        except zipfile.error as e:
            pass

with zipfile.ZipFile(TEST_DATASET_ZIP_DIR, "r") as zip_obj:
    for member in tqdm(zip_obj.infolist(), desc="Extracting "):
        try:
            zip_obj.extract(member, TEST_DATASET_WORKING_DIR)
        except zipfile.error as e:
            pass

In [ ]:
print("After zip extraction:")
print(os.listdir(BOUNCED_DATASET_WORKING_DIR))

In [ ]:
print(os.listdir(DATA_ROOT))
print()
print(os.listdir(TEST_DIR))

In [ ]:
classes_subfolders = [f.path for f in os.scandir(DATA_ROOT) if f.is_dir()]

max_images_count = -1
for moves_folder in classes_subfolders:
    if "say_so_2" not in moves_folder:
        file_list = [f for f in Path(moves_folder).glob("**/*") if f.is_file()]
        if len(file_list) > max_images_count:
            max_images_count = len(file_list)
print("Max images count: {}".format(max_images_count))

In [ ]:
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        if class_name != "say_so_2":
            os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    if class_name != "say_so_2":
        source_dir = os.path.join(DATA_ROOT, class_name)
        for i, file_name in enumerate(tqdm_notebook(os.listdir(source_dir))):
            print(os.path.join(train_dir, class_name))
            if i % 6 != 0:
                dest_dir = os.path.join(train_dir, class_name)
            else:
                dest_dir = os.path.join(val_dir, class_name)
            shutil.copy(
                os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name)
            )

In [ ]:
pie_chart_dict = dict([(key, []) for key in class_names])
for class_sub in classes_subfolders:
    class_in_folder_name = os.path.basename(class_sub)
    if class_in_folder_name != "say_so_2":
        images_count = len(os.listdir(class_sub))
        pie_chart_dict[class_in_folder_name] = images_count

sorted_dict = {}
sorted_keys = sorted(pie_chart_dict, key=pie_chart_dict.get)
for w in sorted_keys:
    sorted_dict[w] = pie_chart_dict[w]
pie_chart_dict = sorted_dict

print(len(pie_chart_dict))

plt.clf()
pie, ax = plt.subplots(figsize=[16, 10])
pie.autolayout = True
pie_chart_labels = [k for k in pie_chart_dict.keys()]
pie_chart_data = [float(v) for v in pie_chart_dict.values()]

# winter / cool / coolwarm / Set3 also can be used for cmap
theme = plt.get_cmap("coolwarm")
ax.set_prop_cycle(
    "color", [theme(1.0 * i / len(pie_chart_data)) for i in range(len(pie_chart_data))]
)

explode = [0.1] * len(class_names)
wedges, labels, autopct = plt.pie(
    x=pie_chart_data,
    labels=pie_chart_labels,
    autopct=lambda p: f"{p:.2f}%,\n{p*sum(pie_chart_data)/100 :.0f} img",
    pctdistance=0.85,
    explode=explode,
    labeldistance=1.1,
    textprops={"fontsize": 9, "color": "black"},
)
[_.set_fontsize(14) for _ in labels]
ax.axis("equal")
plt.tight_layout()
plt.title(
    "Distribution of images in the dataset",
    fontdict={
        "fontsize": 20,
        "weight": "bold",
    },
)

plt.savefig("./{}/distribution_of_images_in_dataset.png".format(RESULT_PLOTS_DIR))
plt.show()
plt.close()

In [ ]:
plt.clf()
fig = go.Figure()
fig.add_trace(go.Pie(labels=pie_chart_labels, values=pie_chart_data))
fig.update_layout(title="Distribution of images in the dataset", template="seaborn")
fig.show()
fig.write_image("./{}/distribution_of_images_in_dataset_2.png".format(RESULT_PLOTS_DIR))

In [ ]:
# output size of one of the images
image = Image.open("train/dab_left/00002.png")
width, height = image.size
print("Source images size: width == {}; height == {}".format(width, height))

In [ ]:
num_of_files = 0
num_of_dir = 0
for base, dirs, files in os.walk(train_dir):
    for directories in dirs:
        num_of_dir += 1
    for Files in files:
        num_of_files += 1

print("Train images count: {}".format(num_of_files))

In [ ]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)


class ImgAugTransform:
    def __init__(self):
        self.aug = seq = iaa.Sequential(
            [
                iaa.Crop(percent=(0, 0.1)),
                iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0, 0.5))),
                sometimes(iaa.LinearContrast((0.65, 1.5))),
                sometimes(iaa.Multiply((0.8, 1.2), per_channel=0.2)),
                sometimes(
                    iaa.Affine(
                        scale={"x": (0.6, 1.2), "y": (0.6, 1.2)},
                        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                        rotate=(-20, 20),
                        shear=(-5, 5),
                    )
                ),
                iaa.SomeOf(
                    (0, 5),
                    [
                        iaa.OneOf(
                            [
                                iaa.GaussianBlur((0, 3.0)),
                                iaa.AverageBlur(k=(2, 7)),
                                iaa.MedianBlur(k=(3, 11)),
                            ]
                        ),
                        iaa.AdditiveGaussianNoise(
                            loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5
                        ),
                        iaa.Invert(0.05, per_channel=True),
                        iaa.Multiply((0.5, 1.5), per_channel=0.5),
                        iaa.LinearContrast((0.5, 2.0), per_channel=0.5),
                        iaa.Grayscale(alpha=(0.0, 1.0)),
                        sometimes(
                            iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
                        ),
                        sometimes(
                            iaa.OneOf(
                                [
                                    iaa.EdgeDetect(alpha=(0, 0.7)),
                                    iaa.DirectedEdgeDetect(
                                        alpha=(0, 0.7), direction=(0.0, 1.0)
                                    ),
                                ]
                            )
                        ),
                    ],
                    random_order=True,
                ),
            ],
            random_order=True,
        )

    def __call__(self, img):
        img = np.array(img).copy()
        return np.ascontiguousarray(self.aug.augment_image(img))

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((IMAGE_CROP_HEIGHT, IMAGE_CROP_WIDTH)),
        ImgAugTransform(),
        transforms.ToTensor(),
    ]
)

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
    pin_memory=True,
)

print("Train dataloader size: {}".format(len(train_dataloader)))

val_transforms = transforms.Compose(
    [transforms.Resize((IMAGE_CROP_HEIGHT, IMAGE_CROP_WIDTH)), transforms.ToTensor()]
)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    pin_memory=True,
)

print("Validation dataloader size: {}".format(len(val_dataloader)))

In [ ]:
def show_input_num(input_tensor, title=""):
    image = input_tensor.permute(1, 2, 0).numpy()
    print(image)


X_batch, y_batch = next(iter(train_dataloader))
for x_item, y_item in zip(X_batch, y_batch):
    show_input_num(x_item, title=class_names[y_item])

In [ ]:
def show_input(input_tensor, title=""):
    image = input_tensor.permute(1, 2, 0).numpy()
    plt.imshow(image)
    plt.title(title)
    plt.show()
    plt.pause(0.001)


X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [ ]:
print(
    "Train dataloader len == {}, train dataset len == {}".format(
        len(train_dataloader), len(train_dataset)
    )
)

# Training and stat

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):

    train_loss_history, valid_loss_history = [], []
    train_accuracy_history, valid_accuracy_history = [], []

    result_log_file = open("train_model_log.txt", "w+")
    result_log_file.truncate(0)
    result_log_file.seek(0)

    for epoch in range(EPOCHS_COUNT):
        epoch_result_str = "Epoch {}/{}:".format(epoch + 1, num_epochs)
        print(epoch_result_str, file=sys.stdout, flush=True)
        print(epoch_result_str, file=result_log_file, flush=True)

        # Each epoch has a training and validation phase
        for phase in ["val", "train"]:
            if phase == "train":
                dataloader = train_dataloader
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_acc = 0.0

            predlist = torch.zeros(len(class_names), dtype=torch.long, device="cpu")
            lbllist = torch.zeros(len(class_names), dtype=torch.long, device="cpu")

            # Iterate over data
            for inputs, labels in tqdm_notebook(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == "train"):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # For confusion matrix
                    predlist = torch.cat(
                        [predlist, preds_class.view(-1).cpu()]
                    )  # Save Prediction
                    lbllist = torch.cat([lbllist, labels.view(-1).cpu()])  # Save Truth

                    # backward + optimize only if in training phase
                    if phase == "train":
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            if phase == "train":
                train_loss_history.append(epoch_loss)
                train_accuracy_history.append(epoch_acc)
            else:
                valid_loss_history.append(epoch_loss)
                valid_accuracy_history.append(epoch_acc)

            loss_acc_result_str = "{} Loss: {:.4f} Acc: {:.4f}".format(
                phase, epoch_loss, epoch_acc
            )
            print(loss_acc_result_str, file=sys.stdout, flush=True)
            print(loss_acc_result_str, file=result_log_file, flush=True)

            # Build confusion matrix
            plt.figure().clear()
            plt.cla()
            plt.clf()

            cf_matrix = confusion_matrix(lbllist.numpy(), predlist.numpy())

            df_cm = pd.DataFrame(
                cf_matrix,
                index=[i for i in class_names],
                columns=[i for i in class_names],
            )
            fig = plt.figure(figsize=(35, 35))

            cm = plt.cm.get_cmap("GnBu")
            sns.heatmap(
                df_cm,
                annot=True,
                vmin=0.0,
                vmax=1.0 * max_images_count,
                fmt=".0f",  # because we use img count
                linewidths=2,
                cmap=cm,
            )

            sns.set(font_scale=1.5)
            plt.title(
                'Confusion Matrix for {}/{} epoch and "{}" phase'.format(
                    epoch + 1, EPOCHS_COUNT, phase
                ),
                fontdict={
                    "fontsize": 20,
                    "weight": "bold",
                },
            )

            plt.xlabel("prediction", fontsize=16)
            plt.ylabel("label (ground truth)", fontsize=16)

            plt.savefig(
                "./{}/{}/epoch_{:04d}_phase_{}.png".format(
                    RESULT_PLOTS_DIR, confusion_matrix_plots_dir, epoch + 1, phase
                )
            )

            plt.close()

    result_log_file.close()

    return (
        model,
        train_loss_history,
        valid_loss_history,
        train_accuracy_history,
        valid_accuracy_history,
    )

In [ ]:
model = models.mobilenet_v2(pretrained=True)

model.classifier = torch.nn.Linear(
    in_features=model.classifier[1].in_features, out_features=len(class_names)
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=1.0e-3)

In [ ]:
model

In [ ]:
(
    model,
    train_loss_history,
    valid_loss_history,
    train_accuracy_history,
    valid_accuracy_history,
) = train_model(model, loss, optimizer, None, num_epochs=EPOCHS_COUNT)

In [ ]:
train_loss_history = torch.tensor(train_loss_history, device="cpu")
valid_loss_history = torch.tensor(valid_loss_history, device="cpu")
train_accuracy_history = torch.tensor(train_accuracy_history, device="cpu")
valid_accuracy_history = torch.tensor(valid_accuracy_history, device="cpu")
dict_data = {
    "Loss Function Train": train_loss_history,
    "Loss Function Valid": valid_loss_history,
    "Accuracy Train": train_accuracy_history,
    "Accuracy Valid": valid_accuracy_history,
}

In [ ]:
df_results = pd.DataFrame.from_dict(dict_data)

In [ ]:
def loss_plot(data):

    result_plots_dir = "plots"
    result_plots_dir_check = "./{}".format(result_plots_dir)
    if not os.path.isdir(result_plots_dir_check):
        os.makedirs(result_plots_dir_check)

    plt.clf()

    loss_names = data.columns[:2]
    accuracy_names = data.columns[2:]
    legend_names = ["Train", "Valid"]

    fig, ax = plt.subplots(1, 2, figsize=(20, 8))

    for i, j, k in zip(loss_names, accuracy_names, legend_names):

        ax[0].plot(data[i].values, label=k)
        ax[1].plot(data[j].values, label=k)

    for i, j in enumerate(["Loss", "Accuracy"]):

        ax[i].set_title(f"{j} Loss Plot", fontsize=14)
        ax[i].set_xlabel("Epoch", fontsize=12)
        ax[i].set_ylabel(f"{j} Loss Function Value", fontsize=12)
        ax[i].legend()

    fig.suptitle("Result of Model Training", fontsize=18)
    plt.savefig("./{}/result_of_model_training.png".format(result_plots_dir))
    plt.show()
    plt.close()

In [ ]:
loss_plot(df_results)

In [ ]:
# All information about model
summary(model)
from torchsummary import summary

summary(model, (3, 256, 160))

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = original_tuple + (path,)
        return tuple_with_path


test_dataset = ImageFolderWithPaths(TEST_DIR, val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

In [ ]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm_notebook(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:, 1].data.cpu().numpy()
    )
    test_img_paths.extend(paths)

test_predictions = np.concatenate(test_predictions)

In [ ]:
submission_df = pd.DataFrame.from_dict(
    {"id": test_img_paths, "label": test_predictions}
)

In [ ]:
submission_df.to_csv("./{}.csv".format(SUMBISSION_FILE_NAME))

In [ ]:
torch.save(model, "./{}.pt".format(MODEL_SAVE_NAME))

## Convert to the tflite model

In [ ]:
# Input to the model
dummy_input = torch.randn(1, 3, IMAGE_CROP_HEIGHT, IMAGE_CROP_WIDTH, requires_grad=True)
dummy_input = dummy_input.to(device)

torch_out = model(dummy_input)

# Export the model
torch.onnx.export(
    model,  # model being run
    dummy_input,  # model input
    "./{}.onnx".format(MODEL_SAVE_NAME),
    input_names=["input"],
    output_names=["output"],
    export_params=True,  # store the trained parameter weights inside the model file
    opset_version=10,  # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    verbose=True,
)

In [ ]:
onnx_model = onnx.load("./{}.onnx".format(MODEL_SAVE_NAME))
try:
    onnx.checker.check_model(onnx_model)
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s" % e)
else:
    print("The model is valid!")

In [ ]:
tf_rep = prepare(onnx_model)

In [ ]:
tf_rep.export_graph(PB_PATH)

In [ ]:
input_nodes = tf_rep.inputs
output_nodes = tf_rep.outputs

print("The names of the input nodes are: {}".format(input_nodes))
print("The names of the output nodes are: {}".format(output_nodes))

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(PB_PATH)
tflite_model = converter.convert()

with open(TF_PATH, "wb") as f:
    f.write(tflite_model)